In [2]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
import os
from pandas import DataFrame

In [3]:
import pandas as pd

optfileList = ['../data/CRex/RHRS/CRex_DataSet_21743.csv',
               '../data/CRex/RHRS/CRex_DataSet_21744.csv',
               '../data/CRex/RHRS/CRex_DataSet_21748.csv',
               '../data/CRex/RHRS/CRex_DataSet_21749.csv',
               '../data/CRex/RHRS/CRex_DataSet_21752.csv',
               '../data/CRex/RHRS/CRex_DataSet_21753.csv',
               '../data/CRex/RHRS/CRex_DataSet_21754.csv',
               '../data/CRex/RHRS/CRex_DataSet_21755.csv',
               '../data/CRex/RHRS/CRex_DataSet_21756.csv',
               '../data/CRex/RHRS/CRex_DataSet_21757.csv',
               '../data/CRex/RHRS/CRex_DataSet_21768.csv',
               '../data/CRex/RHRS/CRex_DataSet_21769.csv',
               '../data/CRex/RHRS/CRex_DataSet_21770.csv',
               '../data/CRex/RHRS/CRex_DataSet_21771.csv',
               '../data/CRex/RHRS/CRex_DataSet_21772.csv',
               '../data/CRex/RHRS/CRex_DataSet_21781.csv',
               '../data/CRex/RHRS/CRex_DataSet_21782.csv',
               '../data/CRex/RHRS/CRex_DataSet_21783.csv',
               '../data/CRex/RHRS/CRex_DataSet_21784.csv',
               '../data/CRex/RHRS/CRex_DataSet_21785.csv',
               ]

data =  pd.read_csv('../data/CRex/RHRS/CRex_DataSet_21743.csv')


In [4]:
selected_feature = ['focal_x','focal_y','focal_th','focal_ph']
targetTheta = ['targCalTh']
targetPhi = ['targCalPh']
data=data[selected_feature + targetTheta + targetPhi]

data

,focal_x,focal_y,focal_th,focal_ph,targCalTh,targCalPh
0,-0.301587,0.016011,0.012192,0.000739,-0.029615,-0.006545
1,-0.037429,-0.007679,0.001400,0.011046,-0.009243,0.008405
2,-0.283216,0.012550,0.004096,-0.000475,-0.016034,-0.006545
3,-0.249758,0.006722,-0.003607,0.006061,-0.002452,-0.000295
4,-0.163223,0.010093,-0.018091,0.009351,0.024711,-0.000295
...,...,...,...,...,...,...
19406,-0.048969,0.027085,0.003012,-0.015183,-0.009243,-0.022169
19407,-0.050548,0.026683,0.002689,-0.015877,-0.009243,-0.022169
19408,-0.097982,0.026277,0.002064,-0.016649,-0.009243,-0.022169
19409,-0.051081,0.026447,0.003396,-0.014656,-0.009243,-0.022169


In [5]:
from SpectroCalib.feature_engineer._preprocess import PolynomialFeatureTransformer
from sklearn.pipeline import Pipeline

In [6]:
fullpipeline = Pipeline([('poly',PolynomialFeatureTransformer(columns = selected_feature))])

transformed = fullpipeline.fit_transform(data)


trans_Theta = transformed[targetTheta]
trans_Phi   = transformed[targetPhi]
trans_X = transformed.drop(targetTheta + targetPhi,axis=1)

## feature selection

In [21]:
#TODO need to move a seperate class

from sklearn.feature_selection import  SelectKBest, f_regression, mutual_info_regression

transformed_theta_feature = SelectKBest(f_regression,k=30).fit(trans_X,trans_Theta)
transformed_phi_feature   = SelectKBest(f_regression,k=30).fit(trans_X,trans_Phi)

theta_index = transformed_theta_feature.get_support()
phi_index = transformed_phi_feature.get_support()

X_train_theta_columns = trans_X.columns[theta_index]
X_train_phi_columns = trans_X.columns[phi_index]


theta_data = trans_X[X_train_theta_columns]
phi_data = trans_X[X_train_phi_columns]

/home/newdriver/Work/research/Calib/venv/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/home/newdriver/Work/research/Calib/venv/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:302: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/home/newdriver/Work/research/Calib/venv/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:307: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/home/newdriver/Work/research/Calib/venv/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **

### AUTO ML implication

Before move forward, let test it with ```auto-sklearn```


#### train test split

In [20]:
import sklearn
X_train_theta, X_test_theta, y_train_theta, y_test_theta = \
            sklearn.model_selection.train_test_split(theta_data, trans_Theta, random_state=1)


X_train_phi, X_test_phi, y_train_phi, y_test_phi = \
            sklearn.model_selection.train_test_split(phi_data, trans_Phi, random_state=1)

#### Step 1.  **train $\theta$**


In [22]:
import autosklearn.regression
import sklearn.model_selection
import sklearn.metrics
import os

automl_theta = autosklearn.regression.AutoSklearnRegressor(n_jobs=os.cpu_count(),tmp_folder='./tmp/autosklearn_regression_example_tmp')

automl_theta.fit(X=X_train_theta,y=y_train_theta,X_test=X_test_theta,y_test=y_test_theta)

[WARNING] [2021-10-01 12:58:01,190:Client-AutoML(1):bbd39ffc-22d8-11ec-ad8f-a3264d3d93e1] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (3599.722542)
[WARNING] [2021-10-01 12:58:01,190:Client-AutoML(1):bbd39ffc-22d8-11ec-ad8f-a3264d3d93e1] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.


KeyboardInterrupt: 